In [ ]:
%cd /hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning/
import torch
import json

eval_instructions = json.load(open("dataset/Arxiv2023_instruction_stage2/instructions.json"))
eval_data = torch.load("dataset/Arxiv2023_instruction_stage2/test_data_list_8192_llama3.pt")

eval_instructions = {
    str(entry["id"]): entry for entry in eval_instructions
}

output_prompts = []
for entry in eval_data:
    id = str(entry["id"])
    truth_i = entry["ground_truth"]

    prompt_i = eval_instructions[id]

    input_i = prompt_i["conversations"][0]["value"]
    input_i = input_i.replace("For each paper, a heterogeneous knowledge graph is constructed to reflect the concepts and key notes in the title and abstract. In the knowledge graph, there are five types of nodes: namely: paper, research_background, research_question, methodology, key_results, keyword. The relationships (meta paths) between different nodes include: [paper has_property research_background], [paper has_property research_question], [paper has_property methodology], [paper has_property key_results], [paper has_keyword keyword], [research_background has_keyword keyword], [research_question has_keyword keyword], [methodology has_keyword keyword], [key_results has_keyword keyword].\n", "")
    input_i_parts = input_i.split("Heterogeneous Graph: \"paper\" node: <graph>;")
    input_i = input_i_parts[0] + "Based on the title, abstract and the provided knowledge graph, which Arxiv category should the paper be categorized into?" + input_i_parts[1].split("Based on the title, abstract and the provided knowledge graph, which Arxiv category should the paper be categorized into?")[1]
    input_i = input_i.replace(" and the provided knowledge graph", "")

    # print(input_i)
    
    output_i = {
        "id": id,
        "prompt": [
            {
            "role": "user",
            "content": input_i
            }
        ],
        "ground_truth": truth_i
    }
    output_prompts.append(output_i)

json.dump(output_prompts, open("dataset/Arxiv2023_instruction_stage2/eval_prompts.json", "w"), indent=4)

In [14]:
%cd /hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning/
import torch
import json
import re

eval_instructions = json.load(open("dataset/ICLR_peer_review_stage2/instructions.json"))
eval_data = torch.load("dataset/ICLR_peer_review_stage2/test_data_list_8192_llama3.pt")

eval_instructions = {
    str(entry["id"]): entry for entry in eval_instructions
}

# use regex to match patterns like \"xxxx <graph>\"
remove_pattern = re.compile(r"\".*?<graph>\"")

output_prompts = []
for entry in eval_data:
    id = str(entry["id"])
    truth_i = entry["ground_truth"]

    prompt_i = eval_instructions[id]

    input_i = prompt_i["conversations"][0]["value"]
    input_i = input_i.replace("For each paper together with its reviews, a heterogeneous knowledge graph is constructed to reflect the concepts and key notes in the whole content. In the knowledge graph, there are several types of nodes: namely: paper, strengths_in_review_n, weaknesses_in_review_n, keyword, with n subject to the number of reviews. The relationships (meta paths) between different nodes include: [paper has_property strengths_in_review_n], [paper has_property weaknesses_in_review_n], [strengths_in_review_n has_keyword keyword], [weaknesses_in_review_n has_keyword keyword], [paper has_keyword keyword].\n", "")
    input_i = input_i.replace(" and the knowledge graph", "")
    input_i = input_i.replace("\"paper\" node in the knowledge graph: <graph>", "")
    input_i = re.sub(remove_pattern, "", input_i)
    # print(input_i)
    # break
    
    output_i = {
        "id": id,
        "prompt": [
            {
            "role": "user",
            "content": input_i
            }
        ],
        "ground_truth": truth_i
    }
    output_prompts.append(output_i)

json.dump(output_prompts, open("dataset/ICLR_peer_review_stage2/eval_prompts.json", "w"), indent=4)

/hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning
